In [ ]:
!pip install -q transformers==4.28.0
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.7 MB/s eta 0:00:00


## Using HuggingFace Dataset class for loading data

In [ ]:
from datasets import load_dataset, load_from_disk

dataset = load_dataset('csv', data_files='/content/IMDB Dataset.csv', split='train')
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-58a6fd4adb03519e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})

In [ ]:
dataset = dataset.rename_column('review', 'text')
dataset = dataset.rename_column('sentiment', 'labels')

In [ ]:
dataset

Dataset({
    features: ['text', 'labels'],
    num_rows: 50000
})

## Using AutoTokenizer for tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
 
encoding_fn = lambda x: 1 if x == 'positive' else 0

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.class_encode_column("labels")

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset, test_dataset = split['train'], split['test']

In [ ]:
train_dataset

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 40000
})

In [ ]:
test_dataset

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

## Using AutoModelForSequenceClassification for classifying text

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

checkpoint = "GroNLP/hateBERT"

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model.config.ctc_zero_infinity = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

if device == 'cude':
  model = model.half()

Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly

In [ ]:
!pip install -q --upgrade accelerate

from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="steps",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='f1'  # This will keep the best model measured by eval f1 score
)
 
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': accuracy, 'f1': f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)
 
trainer.train()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 13.0 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
100,0.368900,0.300257,0.874600,0.874588
200,0.294100,0.285576,0.878600,0.878488
300,0.282100,0.268719,0.888900,0.888728
400,0.196800,0.269262,0.892100,0.892086
500,0.170500,0.285456,0.888500,0.888481
600,0.171200,0.284204,0.893000,0.892999
700,0.116400,0.361088,0.892100,0.892094
800,0.092000,0.351082,0.890700,0.890699
900,0.087600,0.349035,0.892700,0.892701


TrainOutput(global_step=939, training_loss=0.19328416295198841, metrics={'train_runtime': 3163.7066, 'train_samples_per_second': 37.93, 'train_steps_per_second': 0.297, 'total_flos': 7893331660800000.0, 'train_loss': 0.19328416295198841, 'epoch': 3.0})

In [ ]:
trainer.save_model('./best-f1-imdb')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

In [ ]:
shutil.copytree('/content/best-f1-imdb', '/content/drive/MyDrive/best-f1-imdb/hateBERT')

'/content/drive/MyDrive/best-f1-imdb/hateBERT'